# Game Engine

In [ ]:
# | default_exp game.engine

In [ ]:
# | hide
%load_ext lab_black

In [ ]:
# | export
import importlib
import os

All games are located within the `games` subdirectory.

## Engine

In [ ]:
# | export
TITLE_DIR = "engine/game/titles"
TITLE_MODULE = "rl18xx.game.engine.game.title"

In [ ]:
# | export


class Engine:
    def __init__(self):
        self.game_modules = self._import_game_modules()
        self.game_meta = self._collect_game_meta()
        self.games = self._load_games()

    def _import_game_modules(self):
        game_modules = {}
        for filename in os.listdir(TITLE_DIR):
            if filename.endswith(".ipynb"):
                module_name = filename[3:-6]
                module_path = f"{TITLE_MODULE}.{module_name}"
                game_modules[module_name] = importlib.import_module(module_path)
        return game_modules

    def _collect_game_meta(self):
        game_meta_by_title = {}
        for name, module in self.game_modules.items():
            if hasattr(module, "Meta"):
                meta = getattr(module, "Meta")
                game_meta_by_title[meta.title()] = meta
        return game_meta_by_title

    def _load_games(self):
        games = {}
        for title in self.game_meta:
            if hasattr(self.game_modules["g" + title], "Game"):
                games[title] = getattr(self.game_modules["g" + title], "Game")
        return games

    def meta_by_title(self, title):
        return self.game_meta.get(title)

    def game_by_title(self, title):
        return self.games.get(title)

In [ ]:
engine = Engine()

In [ ]:
game = engine.game_by_title("1830")

## Game Instantiation and basic tests

In [ ]:
g = game({"1": "hi", "2": "my", "3": "dear", "4": "friend"})

In [ ]:
import inspect

test = False

skipped = []
methods = []
non_methods = []
for item_name in dir(g):
    if item_name.startswith("init") or item_name.startswith("_") or item_name.isupper():
        skipped.append(item_name)
        continue

    item = getattr(g, item_name)
    if callable(item):
        if inspect.signature(item).parameters:
            skipped.append(item)
        else:
            # print(item_name)
            methods.append(item_name)
            if test:
                item()
    else:
        # print(f"name: {item_name}, value: {item}")
        non_methods.append(item_name)

# methods, non_methods

## Actions Helper

In [ ]:
from rl18xx.game.engine.actions import (
    Bid,
    BuyShares,
    BuyTrain,
    LayTile,
    Par,
    Pass,
    PlaceToken,
    SellShares,
)

from rl18xx.game.engine.round import BuyTrain as BuyTrainStep

from collections import defaultdict


def get_current_actions():
    return g.active_step().actions(g.current_entity)


def get_bid_actions():
    companies = g.active_step().companies
    bids = []
    bids.append(
        Bid(g.current_entity, g.active_step().min_bid(companies[0]), companies[0])
    )
    for company in companies[1:]:
        min_bid = g.active_step().min_bid(company)
        max_bid = g.active_step().max_bid(g.current_entity, company)
        bid_values = list(
            range(min_bid - (min_bid % 5), (max_bid + 1) - ((max_bid + 1) % 5), 5)
        )
        bids.append(
            [Bid(g.current_entity, bid_value, company) for bid_value in bid_values]
        )
    return bids


def get_par_actions():
    parable_corporations = sorted(
        [corp for corp in g.corporations if g.can_par(corp, g.current_entity)],
        key=lambda corporation: corporation.name,
    )
    par_values = g.share_prices
    buying_power = g.buying_power(g.current_entity)
    return [
        Par(g.current_entity, corp, price)
        for corp in parable_corporations
        for price in par_values
        if 2 * price.price <= buying_power
    ]


def get_buy_shares_actions():
    buyable_shares = [
        item
        for sublist in g.active_step().buyable_shares(g.current_entity)
        for item in sublist
    ]
    unique_buyable_shares = sorted(
        list(set(buyable_shares)), key=lambda share: share.corporation()
    )
    return [BuyShares(g.current_entity, share) for share in unique_buyable_shares]


def get_sell_shares_actions():
    if isinstance(g.active_step(), BuyTrainStep):
        return []
    else:
        return g.active_step().sellable_shares(g.current_entity)


def get_place_token_actions():
    if hasattr(g.active_step(), "pending_token") and g.active_step().pending_token:
        hexes = g.active_step().pending_token.get("hexes")
        return [
            PlaceToken(g.current_entity, city, city.get_slot(g.current_entity))
            for hex in hexes
            for city in hex._tile.cities
        ]

    return [
        PlaceToken(g.current_entity, city, city.get_slot(g.current_entity))
        for city in g.graph.connected_nodes(g.current_entity)
        if city.tokenable(g.current_entity)
    ]


def get_lay_tile_actions():
    # TODO: May need this in later phases
    # upgradable_tiles = {}
    # upgradable_tiles[hex] = g.active_step().upgradeable_tiles(g.current_entity, hex)
    moves = defaultdict(dict)
    for hex in g.graph.connected_hexes(g.current_entity):
        layable_tiles = g.active_step().potential_tiles(g.current_entity, hex)
        for tile in layable_tiles:
            rotations = g.active_step().legal_tile_rotations(
                g.current_entity, hex, tile
            )
            moves[hex][tile] = rotations
    return [
        LayTile(g.current_entity, tile, hex, rotation)
        for hex in moves
        for tile in moves[hex]
        for rotation in moves[hex][tile]
    ]

def get_buy_train_actions():
    # TODO: implement <buy cheapest train from depot>
    # TODO: implement <buy any train from share/train pool>
    # TODO: implement <buy any train for any price from corps you own>
    pass


def get_choices_for_action(action):
    if action == Pass:
        return [Pass(g.current_entity)]
    elif action == Bid:
        return get_bid_actions()
    elif action == Par:
        return get_par_actions()
    elif action == BuyShares:
        return get_buy_shares_actions()
    elif action == SellShares:
        return get_sell_shares_actions()
    elif action == PlaceToken:
        return get_place_token_actions()
    elif action == LayTile:
        return get_lay_tile_actions()
    elif action == BuyTrain:
        return get_buy_train_actions()
    else:
        return []


def get_all_choices():
    choices = [
        choices
        for action in get_current_actions()
        for choices in get_choices_for_action(action)
    ]
    return choices


# get_all_choices()

In [ ]:
step = g.active_step()
step, g.active_players()
print(f"step: {g.active_step()}, player: {g.current_entity}")
choices = g.active_step().actions(g.current_entity)
print(f"choices: {choices}")

step: <WaterfallAuction>, player: Player - hi
choices: [<class 'rl18xx.game.engine.actions.Bid'>, <class 'rl18xx.game.engine.actions.Pass'>]


## Waterfall auction

In [ ]:
# get_all_choices()

In [ ]:
pass_ = choices[1](g.active_players()[0])
g.process_action(pass_)

second = g.round.active_step().companies[1]
bid = choices[0](
    g.active_players()[0], g.round.active_step().min_bid(second), company=second
)
g.process_action(bid)

bid = choices[0](
    g.active_players()[0], g.round.active_step().min_bid(second), company=second
)
g.process_action(bid)

later = g.round.active_step().companies[-1]
bid = choices[0](g.active_players()[0], 225, company=later)
g.process_action(bid)

first = g.round.active_step().companies[0]
bid = choices[0](
    g.active_players()[0], g.round.active_step().min_bid(first), company=first
)
g.process_action(bid)

pass_ = choices[1](g.active_players()[0])
g.process_action(pass_)

first = g.round.active_step().companies[0]
bid = choices[0](
    g.active_players()[0], g.round.active_step().min_bid(first), company=first
)
g.process_action(bid)

while g.round.active_step() == step:
    first = g.round.active_step().companies[0]
    bid = choices[0](
        g.active_players()[0], g.round.active_step().min_bid(first), company=first
    )
    g.process_action(bid)

par = g.round.active_step().actions(g.round.active_step().active_entities[0])[0](
    g.round.active_step().active_entities[0],
    g.round.companies_pending_par[0].abilities[3].shares[0].corporation(),
    g.round.active_step().get_par_prices(
        g.round.active_step().active_entities[0],
        g.round.companies_pending_par[0].abilities[3].shares[0].corporation,
    )[0],
)

g.process_action(par)

<rl18xx.game.engine.game.title.g1830.Game>

## Stock Round 1

In [ ]:
g.process_action(get_all_choices()[-2])
g.process_action(get_all_choices()[-1])
g.process_action(get_all_choices()[-8])
g.process_action(get_all_choices()[1])
g.process_action(get_all_choices()[1])
g.process_action(get_all_choices()[14])
g.process_action(get_all_choices()[2])
g.process_action(get_all_choices()[1])
g.process_action(get_all_choices()[1])
g.process_action(get_all_choices()[3])
g.process_action(get_all_choices()[2])
g.process_action(get_all_choices()[0])
g.process_action(get_all_choices()[1])
g.process_action(get_all_choices()[3])
g.process_action(get_all_choices()[2])
g.process_action(get_all_choices()[1])
g.process_action(get_all_choices()[2])
g.process_action(get_all_choices()[1])
g.process_action(get_all_choices()[1])
g.process_action(get_all_choices()[2])
g.process_action(get_all_choices()[1])
g.process_action(get_all_choices()[1])

<rl18xx.game.engine.game.title.g1830.Game>

## Operating Round 1

In [ ]:
g.process_action(get_all_choices()[0])
g.process_action(get_all_choices()[1])

<rl18xx.game.engine.game.title.g1830.Game>

In [ ]:
print(
    f"step: {g.active_step()}, player: {g.current_entity}, purchasing power: {g.buying_power(g.current_entity)}"
)
print(f"possible actions: {g.active_step().actions(g.current_entity)}")
get_all_choices()

step: <BuyTrain>, player: <Corporation: PRR>, purchasing power: 670.0
possible actions: [<class 'rl18xx.game.engine.actions.SellShares'>, <class 'rl18xx.game.engine.actions.BuyTrain'>]


[]

## TODO

* figure out why run routes and pay dividends were not automatically skipped and auto-skip it
    * think about how to do the route runner - probably auto route everything to make it easier instead of making the AI do it
    * in OR 2 we will implement run-routes and dividend step actions
* next step to handle should be BuyTrain though

In [ ]:
isinstance(g.active_step(), BuyTrainStep)

True

In [ ]:
g.log

[<Entry message='-- Phase 2 (Operating Rounds: 1 | Train Limit: 4 | Available Tiles: Yellow)', action_id=0>,
 <Entry message='adding share <Share: PRR 10%> for id PRR_1 to ability <rl18xx.game.engine.abilities.Shares object>', action_id=0>,
 <Entry message='setting <rl18xx.game.engine.abilities.Shares object>'s shares to [<Share: PRR 10%>]', action_id=0>,
 <Entry message='adding share <Share: B&O 20%> for id B&O_0 to ability <rl18xx.game.engine.abilities.Shares object>', action_id=0>,
 <Entry message='setting <rl18xx.game.engine.abilities.Shares object>'s shares to [<Share: B&O 20%>]', action_id=0>,
 <Entry message='hi passes bidding', action_id=1>,
 <Entry message='my bids $45 for Champlain & St.Lawrence', action_id=2>,
 <Entry message='dear bids $50 for Champlain & St.Lawrence', action_id=3>,
 <Entry message='friend bids $225 for Baltimore & Ohio', action_id=4>,
 <Entry message='hi buys Schuylkill Valley for $20', action_id=5>,
 <Entry message='Champlain & St.Lawrence goes up for auc